In [15]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
from scipy.stats import ttest_ind
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
import numpy as np
import seaborn as sns
from scipy.stats import pointbiserialr
from torcheeg.models import EEGNet
from torcheeg.trainers import ClassifierTrainer
import pytorch_lightning as pl
from torcheeg.model_selection import KFoldGroupbyTrial

import sys
sys.path.insert(0, 'D:/Navid/Projects2/lexical_memory_task_eeg/')

from src.settings import Paths, Settings
from src.data_loader import VerbMemEEGDataLoader, PilotEEGDataLoader, CLEARDataLoader, TorchEEGDataset
from src.data_preprocess import DataPreprocessor
from src.visualization.EEG_vissualizer import visualize_erp, visualize_block_ERP, visualize_feature_box_plot, \
    visualize_block_features, visualize_p_value_barplots
from src.data_loader import EEGDataSet
from src.feature_extraction import FeatureExtractor



In [16]:
# Load settings from settings.json
settings = Settings()  # Initialize settings object
settings.load_settings()  # Load settings from a JSON file
use_groups = False

# Set up paths for data
paths = Paths(settings)  # Initialize paths object with loaded settings
paths.load_device_paths()  # Load device-specific paths
paths.create_paths()  # Create any necessary file paths

In [18]:
# Load EEG dataset using configured settings and paths
dataset = PilotEEGDataLoader(paths=paths, settings=settings)  # Initialize EEG dataset loader
dataset.load_data(patient_ids=settings.patient)  # Load EEG data for specified patients

# Preprocess the loaded dataset
data_preprocessor = DataPreprocessor(paths=paths, settings=settings)  # Initialize data preprocessor
dataset = data_preprocessor.preprocess(dataset)  # Apply preprocessing steps to the dataset


In [19]:
single_patient_data = dataset.all_patient_data[list(dataset.all_patient_data.keys())[0]]

dataset_train = TorchEEGDataset(data=single_patient_data.data[:-80], labels=single_patient_data.labels['is_experienced'][:-80])
dataset_test = TorchEEGDataset(data=single_patient_data.data[-80:], labels=single_patient_data.labels['is_experienced'][-80:])

train_loader = DataLoader(dataset_train, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset_test, batch_size=64, shuffle=False)

In [21]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 64), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(4*2*7, 1)
        

    def forward(self, x):
        # Layer 1
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)
        
        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)
        
        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)
        
        # FC Layer
        x = x.view(-1, 4*2*7)
        x = F.sigmoid(self.fc1(x))
        return x


net = EEGNet().cuda(0)
print(net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 120, 64)).cuda(0))))
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters())